# Метод пристрелки

In [7]:
import numpy as np
from scipy.integrate import solve_ivp

In [8]:
def r2bp(t, y):
    rvect = y[0:3].copy()
    vvect = y[3:6].copy()
    r = np.linalg.norm(rvect)
    dyd0 = np.reshape(y[6:42].copy(), (6, 6))
    dfdy = np.zeros((6, 6))
    dfdy[0:3, 3:6] = np.eye(3)
    dfdy[3:6, 0:3] = 3*np.outer(rvect, rvect)/r**5 - np.eye(3)/r**3 
    dydt = np.empty(42)
    dydt[0:3] = vvect
    dydt[3:6] = -rvect/r**3
    dydt[6:42] = np.reshape(dfdy @ dyd0, (36,))
    dydt = np.append(vvect, -rvect/np.linalg.norm(rvect)**3)
    return dydt

In [13]:
def residue(x, t0, tf, R0, Rf):
    y0 = np.concatenate((R0, x, np.reshape(np.eye(6), (36,))))
    sol = solve_ivp(r2bp, [t0, tf], y0, method='RK45', atol=1e-06, rtol=1e-06)
    rf = sol.y[0:3,-1]
    dyfdy0 = np.reshape(sol.y[6:42, -1], (6, 6))
    jac = dyfdy0[0:3, 3:6]
    f = rf - Rf
    return f, jac

In [16]:
def newton(func, x0, maxiter=100):
    iter = 0
    while True:
        iter += 1
        f0, jac0 = func(x0)
        if np.linalg.norm(f0) < 1e-10:
            flag = 0
            return x0, f0, flag
        elif iter == maxiter:
            flag = 1
            return x0, f0, flag
        dx = np.linalg.solve(jac0, f0)
        x1 = x0 - dx
        x0 = x1.copy()

In [17]:
t0 = 0.0
tf = np.pi/2
R0 = np.array([1.0, 0.0, 0.0])
Rf = np.array([0.0, 1.0, 0.0])
x0 = np.array([1/2, 1/2, 0.0])
func = lambda x : residue(x, t0, tf, R0, Rf)
sol = newton(func, x0)

ValueError: operands could not be broadcast together with shapes (6,) (42,) 

In [8]:
sol

NameError: name 'sol' is not defined